In [ ]:
import numpy as np
import cv2
from collections import Counter
import heapq

DCT of 8x8 2D Matrix

In [ ]:
def alpha(k):
    return 1 / np.sqrt(2) if k == 0 else 1

def dct_2d_block(block):
    N = block.shape[0]  # Assuming block is N x N (here, N = 8)
    dct_block = np.zeros((N, N), dtype=float)
    for u in range(N):
        for v in range(N):
            sum_val = 0.0
            for x in range(N):
                for y in range(N):
                    sum_val += block[x, y] * np.cos((2 * x + 1) * u * np.pi / (2 * N)) * np.cos((2 * y + 1) * v * np.pi / (2 * N))
            dct_block[u, v] = 0.25 * alpha(u) * alpha(v) * sum_val
    return dct_block

RGB to YCbCr

In [ ]:
def rgb_to_ycbcr(image):
    img_array = np.array(image, dtype=np.float32)
    transformation_matrix = np.array([
        [0.299, 0.587, 0.114],
        [-0.168736, -0.331264, 0.5],
        [0.5, -0.418688, -0.081312]
    ])
    offset = np.array([0, 128, 128])
    ycbcr = np.dot(img_array, transformation_matrix.T) + offset
    return ycbcr.clip(0, 255).astype(np.uint8)

Image into 8x8 Sub-blocks

In [ ]:
def Divide_into_8by8_blocks(img):
  blocks = []
  for i in range(0,img.shape[0]//8,8):
    for j in range(0,img.shape[1]//8,8):
      blocks.append(img[i*8:i*8+8,j*8:j*8+8])
  return np.array(blocks)

VLI and DC Codes

In [ ]:
def Get_VLI_code(Y_zg,VLI_code,DC_VLI_code):
  Y_encoded = []
  prev_DC = -4
  for i in range(len(Y_zg)):
    code = []
    code.append((Y_zg[i][0]-prev_DC))
    DC_VLI_code.append((Y_zg[i][0]-prev_DC))
    prev_DC = Y_zg[i][0]
    Zero_leng = 0
    for j in range(1,len(Y_zg[i])):
        if(Y_zg[i][j]==0):
          Zero_leng+=1
        else:
          cate = 0
          for k in range(1,10):
            if(abs(Y_zg[i][j])>=pow(2,k-1) and abs(Y_zg[i][j])<pow(2,k)):
              cate = k
              break
          code.append(((Zero_leng,cate),Y_zg[i][j]))
          VLI_code.append((Zero_leng,cate))
          Zero_leng = 0
    if(Zero_leng!=0):
      code.append(((Zero_leng,0),0))
      VLI_code.append((Zero_leng,0))
    code.append(((0,0)))
    VLI_code.append((0,0))
    Y_encoded.append(code)
  return Y_encoded

Zig Zag Traversal

In [ ]:
def Zig_Zag(mat):
  d = {}
  res = []
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      if(i+j in d.keys()):
        d[i+j].append(mat[i,j])
      else:
        d[i+j] = [mat[i,j]]
  for i in range(14):
    if i in d:
      l = d[i]
      if(i%2==0):
        l.reverse()
      for ele in l:
        res.append(ele)
  return res

Tree For Making Nodes To Represents Each Symbols

In [ ]:
class Node:
    def __init__(self, freq, symbol=None, left=None, right=None):
        self.freq = freq  # Frequency of the symbol
        self.symbol = symbol  # Symbol (None for internal nodes)
        self.left = left  # Left child
        self.right = right  # Right child

    def __lt__(self, other):
        return self.freq < other.freq

class HuffmanCoding:
    def __init__(self, frequencies):
        self.frequencies = frequencies
        self.root = None

    # Build the Huffman Tree
    def build_tree(self):
        heap = [Node(freq, symbol) for symbol, freq in self.frequencies.items()]
        heapq.heapify(heap)
        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)
            merged = Node(left.freq + right.freq, left=left, right=right)
            heapq.heappush(heap, merged)
        self.root = heap[0]  # Root of the Huffman Tree
        return self.root

    # Generate Huffman Codes by traversing the tree
    def generate_codes(self, node=None, prefix="", codebook=None):
        if codebook is None:
            codebook = {}
        if node is None:
            node = self.root

        if node.symbol is not None:
            codebook[node.symbol] = prefix
        else:
            self.generate_codes(node.left, prefix + "0", codebook)
            self.generate_codes(node.right, prefix + "1", codebook)
        return codebook

# --- Function to encode data using Huffman codes ---
def encode_data(data, huffman_codes):
    encoded_data = ""
    for value in data:
        # Check if the value is in the Huffman codes dictionary
        if value in huffman_codes:
            encoded_data += huffman_codes[value]  # Replace value with its code
        else:
            # Handle the case where the value is not found (e.g., skip, error message)
            print(f"Warning: Value {value} not found in Huffman codes.")

In [ ]:
# Example usage
image = cv2.imread("/content/inputs/mandril_color.jpg")
h, w, _= image.shape
img = image[:,:,::-1]
X_padd = image.shape[0]%8
Y_padd = image.shape[1]%8

if X_padd != 0 :
  image = np.pad(image, ((0, 8-X_padd), (0, 0), (0, 0)), mode='symmetric')
if Y_padd != 0:
  image = np.pad(image, ((0, 0), (0, 8-Y_padd), (0, 0)), mode='symmetric')


Quantization

In [ ]:
ycbcr_image = rgb_to_ycbcr(image)
y = ycbcr_image[:,:,0]
cb = ycbcr_image[:,:,1]
cr = ycbcr_image[:,:,2]
y_blocks = Divide_into_8by8_blocks(y)
cb_blocks = Divide_into_8by8_blocks(cb)
cr_blocks = Divide_into_8by8_blocks(cr)

cb_blocks = cb_blocks[::1,::2,::2]
cr_blocks = cr_blocks[::1,::2,::2]

for i in range(y_blocks.shape[0]):
  y_blocks[i] = dct_2d_block(y_blocks[i])

for i in range(cb_blocks.shape[0]):
  cb_blocks[i] = dct_2d_block(cb_blocks[i])
  cr_blocks[i] = dct_2d_block(cr_blocks[i])

  Y_quantization_table = np.array([
    [16, 11, 10, 16, 24,  40,  51,  61],
    [12, 12, 14, 19, 26,  58,  60,  55],
    [14, 13, 16, 24, 40,  57,  69,  56],
    [14, 17, 22, 29, 51,  87,  80,  62],
    [18, 22, 37, 56, 68, 109, 103,  77],
    [24, 35, 55, 64, 81, 104, 113,  92],
    [49, 64, 78, 87, 103, 121, 120, 101],
    [72, 92, 95, 98, 112, 100, 103,  99]
])
C_quantization_table = np.array([
    [17, 18, 24, 47, 99, 99, 99, 99],
    [18, 21, 26, 66, 99, 99, 99, 99],
    [24, 26, 56, 99, 99, 99, 99, 99],
    [47, 66, 99, 99, 99, 99, 99, 99],
    [99, 99, 99, 99, 99, 99, 99, 99],
    [99, 99, 99, 99, 99, 99, 99, 99],
    [99, 99, 99, 99, 99, 99, 99, 99],
    [99, 99, 99, 99, 99, 99, 99, 99]
])

for i in range(y_blocks.shape[0]):
  for j in range(y_blocks.shape[1]):
    for k in range(y_blocks.shape[2]):
      y_blocks[i,j,k] = np.round(y_blocks[i,j,k]/Y_quantization_table[j,k])

for i in range(cb_blocks.shape[0]):
  for j in range(cb_blocks.shape[1]):
    for k in range(cb_blocks.shape[2]):
      cb_blocks[i,j,k] = np.round(cb_blocks[i,j,k]/C_quantization_table[j,k])
      cr_blocks[i,j,k] = np.round(cr_blocks[i,j,k]/C_quantization_table[j,k])

Y_zg = []
Cb_zg = []
Cr_zg = []

for i in range(y_blocks.shape[0]):
  Y_zg.append(Zig_Zag(y_blocks[i]))
  Cb_zg.append(Zig_Zag(cb_blocks[i]))
  Cr_zg.append(Zig_Zag(cr_blocks[i]))

Y_zg = np.array(Y_zg).astype(int)
Cb_zg = np.array(Cb_zg).astype(int)
Cr_zg = np.array(Cr_zg).astype(int)

VLI_code = []
DC_VLI_code = []

Y_encoded = Get_VLI_code(Y_zg,VLI_code,DC_VLI_code)
Cr_encoded = Get_VLI_code(Cr_zg,VLI_code,DC_VLI_code)
Cb_encoded = Get_VLI_code(Cb_zg,VLI_code,DC_VLI_code)

# print(Y_encoded)
# print(Cr_encoded)
# print(Cb_encoded)

# print(VLI_code)
VLI_freq = Counter(VLI_code)

huffman_AC = HuffmanCoding(VLI_freq)
huffman_AC.build_tree()
VLI_huff_codes = huffman_AC.generate_codes()
huffman__DC = HuffmanCoding(Counter(DC_VLI_code))
huffman__DC.build_tree()
VLI_DC_huff_codes = huffman__DC.generate_codes()

# print(VLI_huff_codes)
# print(VLI_DC_huff_codes)

# Concatenate Huffman codes for Y_encoded, Cr_encoded, and Cb_encoded into separate strings
Y_huffman_encoded_string = ""
Cr_huffman_encoded_string = ""
Cb_huffman_encoded_string = ""

# Loop through each block in Y_encoded, Cr_encoded, and Cb_encoded and concatenate them
for block in Y_encoded:
    dc_value = block[0]
    dc_encoded = VLI_DC_huff_codes.get(dc_value, "")
    ac_encoded = "".join(VLI_huff_codes.get(ac_value, "") for ac_value in block[1:])
    Y_huffman_encoded_string += dc_encoded + ac_encoded

for block in Cr_encoded:
    dc_value = block[0]
    dc_encoded = VLI_DC_huff_codes.get(dc_value, "")
    ac_encoded = "".join(VLI_huff_codes.get(ac_value, "") for ac_value in block[1:])
    Cr_huffman_encoded_string += dc_encoded + ac_encoded

for block in Cb_encoded:
    dc_value = block[0]
    dc_encoded = VLI_DC_huff_codes.get(dc_value, "")
    ac_encoded = "".join(VLI_huff_codes.get(ac_value, "") for ac_value in block[1:])
    Cb_huffman_encoded_string += dc_encoded + ac_encoded

# Print the results to check
print("Y_encoded Huffman string:\n", Y_huffman_encoded_string)
print("Cr_encoded Huffman string:\n", Cr_huffman_encoded_string)
print("Cb_encoded Huffman string:\n", Cb_huffman_encoded_string)

# Write the encoded data and Huffman codes to a text file
with open("encoded_image_data.txt", "w") as f:
    # Write Y, Cr, and Cb encoded data
    f.write("Y_encoded:\n")
    for item in Y_encoded:
        f.write(f"{item}\n")

    f.write("\nCr_encoded:\n")
    for item in Cr_encoded:
        f.write(f"{item}\n")

    f.write("\nCb_encoded:\n")
    for item in Cb_encoded:
        f.write(f"{item}\n")

    # Write Huffman codes for VLI (AC) and DC
    f.write("\nVLI_huff_codes:\n")
    for symbol, code in VLI_huff_codes.items():
        f.write(f"{symbol}: {code}\n")

    f.write("\nVLI_DC_huff_codes:\n")
    for symbol, code in VLI_DC_huff_codes.items():
        f.write(f"{symbol}: {code}\n")

print("Encoded data written to text file successfully.")

Y_encoded Huffman string:
 11000111010011000001001001001001110010000101001101100100000010011100100100100100010000100010110100100000010001000010001001010011000001000010100101101010010011010010001010001101001000010100111101010010000001001111000100101001000000100110110010000001000010100111110100110101010011100100110100010001010010001101001101000100101001001000101001010010010010010011011110100101001001111010100111110100110110010011010001001111010100110111001000101101000000100111101010011011100100110001100100100001010001101001101010100111110100000010001000010011010001000110100001010011110001001101010100
Cr_encoded Huffman string:
 110001010100000010010011010000101000110100011010001101001110010000001000110100100110100001010011100100111001000100101001101010100011010000001001010010010001010010110101001111000100111110100111001000010100111001001111000100111001001001001000101001001111101000110100010110100101001000101001000000100110010100101110100110010100000010001011010001001010010001010000101001